FUNCTIONS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import random as rnd
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
import warnings
import matplotlib.pyplot as plt
import numpy as np
import itertools
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from matplotlib import pyplot
import matplotlib
warnings.simplefilter(action='ignore', category=FutureWarning)


def getDataset(small,lenght,default=0):
    dataset_path = 'https://raw.githubusercontent.com/TM111/Banking-Dataset_Marketing-Targets_Dataspaces-Exam/main/bank_dataset_full.csv'
    if(small==1):
        dataset_path = 'https://raw.githubusercontent.com/TM111/Banking-Dataset_Marketing-Targets_Dataspaces-Exam/main/bank_dataset_small.csv'
        n = len(pd.read_csv(dataset_path,delimiter=";"))
        skip=[]
        if(n-lenght>1):
            skip = sorted(rnd.sample(range(1,n+1),n-lenght)) 
        dataset = pd.read_csv(dataset_path,delimiter=";", skiprows=skip)
    else:
        dataset=pd.read_csv(dataset_path,delimiter=";")

    dataset.drop(["duration","month","day_of_week"], axis=1, inplace=True) # elimino attributi
    if(default==0):
      dataset.drop(["default"], axis=1, inplace=True) # elimino default
      
    dataset=deleteMissingValues(dataset, "unknown")
    print("\nDataset size "+str(len(dataset))+"\n")
    
    for i in range(2,rnd.randint(3,6)):#mescolo un po il dataset
        for j in range(2,rnd.randint(3,6)):
          dataset = dataset.sample(frac=1).reset_index(drop=True) 
    return dataset
 
def monthToSeason(ds):
    d = {"dec": "autumn_winter","jan": "autumn_winter","feb": "autumn_winter","mar": "spring_summer","apr": "spring_summer","may": "spring_summer",
         "jun": "spring_summer","jul": "spring_summer","aug": "spring_summer","sep": "autumn_winter","oct": "autumn_winter","nov": "autumn_winter",}
    for i in range(0,len(ds)):
        ds.at[i,"month"]=d[ds.iloc[i]["month"]]
    return ds
    
    
def deleteMissingValues(ds,att):
    dataset=ds
    indexRows=[]
    index=-1
    for row in dataset.iloc:
        index=index+1
        percentage="Delete missing values: "+str(int(100*index/len(dataset)))+"%"
        sys.stdout.write('\r'+percentage)
        if("unknown" in row.values):
            indexRows.append(index)
    sys.stdout.write('\r'+"                                             "+'\r')
    dataset.drop(indexRows , inplace=True)  #elimino missing values
    dataset.reset_index(drop=True, inplace=True) 
    return dataset
 
def encodingDataset(dataset):
    dataset=labelEncoder(dataset,["housing","loan","contact","y"])
    dataset=OneHotEncoder(dataset,["job","marital","education","poutcome"]) 
    return dataset
    
    
def OneHotEncoder(ds,attributes):
    dataset=ds
    for att in attributes:
        one_hot = pd.get_dummies(dataset[att])
        dataset = dataset.drop(att,axis = 1)
        dataset = dataset.join(one_hot)
        for i in range(0,len(one_hot.columns)):
            dataset = dataset.rename(columns={one_hot.columns[i]: att+"_"+one_hot.columns[i]})
    attributes=[]
    for att in dataset.columns:
        if(att!="y"):
            attributes.append(att)
    attributes.append("y")
    dataset= dataset[attributes]
    return dataset
 
def labelEncoder(ds,attributes):
    dataset=ds
    le = LabelEncoder()
    for att in attributes:
        dataset[att]=le.fit(dataset[att]).transform(dataset[att])
    return dataset
 
def getOccurrences(ds,attribute,normalize=0,order=0):
    column=ds[attribute]
    values={}
    for v in column:
        if(v not in values):
            if(attribute=="euribor3m"):
                v=str(round(float(v),2))
            values[v]=[0,0]
    
    for i in range(0,len(ds)):
        value=ds.iloc[i][attribute]
        c=ds.iloc[i]["y"]
        if(attribute=="euribor3m"):
                value=str(round(float(value),2))
        if(c=="no" or c==0):
            values[value]=[values[value][0]+1,values[value][1]]
        else:
            values[value]=[values[value][0],values[value][1]+1]
 
    keys=list(values.keys())
 
    no_list=[]
    yes_list=[]
    for k in values.keys():
        sum=values[k][0]+values[k][1]
        if(normalize==0):
            sum=1
        no_list.append(values[k][0]/sum)
        yes_list.append(values[k][1]/sum)
    if(order==1):
        if(normalize==1):
            for i in range(0,len(yes_list)-1):
                for j in range(0,len(yes_list)-1):
                    if(yes_list[j]>yes_list[j+1]):
                        yes_list[j], yes_list[j+1] = yes_list[j+1], yes_list[j]
                        no_list[j], no_list[j+1] = no_list[j+1], no_list[j]
                        keys[j], keys[j+1] = keys[j+1], keys[j]
        else:
            for i in range(0,len(yes_list)-1):
                for j in range(0,len(yes_list)-1):
                    if(yes_list[j]+no_list[j]<yes_list[j+1]+no_list[j+1]):
                        yes_list[j], yes_list[j+1] = yes_list[j+1], yes_list[j]
                        no_list[j], no_list[j+1] = no_list[j+1], no_list[j]
                        keys[j], keys[j+1] = keys[j+1], keys[j]
    if(order==2):
        for i in range(0,len(yes_list)-1):
            for j in range(0,len(yes_list)-1):
                a=keys[j].split("\n")[0]
                b=keys[j+1].split("\n")[0]
                try:
                    if(float(a)>float(b)):
                        yes_list[j], yes_list[j+1] = yes_list[j+1], yes_list[j]
                        no_list[j], no_list[j+1] = no_list[j+1], no_list[j]
                        keys[j], keys[j+1] = keys[j+1], keys[j]
                except:
                    if(a>b):
                        yes_list[j], yes_list[j+1] = yes_list[j+1], yes_list[j]
                        no_list[j], no_list[j+1] = no_list[j+1], no_list[j]
                        keys[j], keys[j+1] = keys[j+1], keys[j]
    return keys,no_list,yes_list
    
def getStatistic(ds,attribute):
    column=ds[attribute]
    statistics={}
    statistics["Min"]=min(column)
    statistics["1st Q."]=np.quantile(column, 0.25)
    statistics["Median"]=np.quantile(column, 0.5)
    statistics["Mean"]=(sum(column)/len(column))
    statistics["Std"]=np.std(column, axis=0)
    statistics["3st Q."]=np.quantile(column, 0.75)
    statistics["Max"]=max(column)
    return statistics  
    
def drawStatisticsTable(data,columns,rows):
    matplotlib.rcParams.update(matplotlib.rcParamsDefault)
    data_tmp=np.array(data).T.tolist()
    for i in range(0,len(rows)):
        for j in range(0,len(columns)):
            if(int(data_tmp[i][j])==data_tmp[i][j]):
                data_tmp[i][j]=int(data_tmp[i][j])
            else:
                data_tmp[i][j]=round(data_tmp[i][j],3)
    Rcolors = plt.cm.BuPu(np.linspace(0.3, 0.3, len(rows)))
    Ccolors = plt.cm.BuPu(np.linspace(0.3, 0.3, len(columns)))
    cell_text = []
    for row in data_tmp:
        cell_text.append(row)
    the_table = plt.table(cellText=cell_text,
                      rowLabels=rows,
                      rowColours=Rcolors,
                      colColours=Ccolors,
                      colLabels=columns,
                      cellLoc="center",
                      loc='center')
    the_table.scale(1.2, 3)
    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.box(on=None)
    plt.show()

def plot_cm(cm,target_names,title='Confusion matrix',cmap=None,normalize=False):
    matplotlib.rcParams.update(matplotlib.rcParamsDefault)
    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy
    precision = round(float(cm.item(3))/float((cm.item(1)+cm.item(3))),2)
    recall=round(float(cm.item(3))/float((cm.item(2)+cm.item(3))),2)
    if cmap is None:
        cmap = plt.get_cmap('Blues')
    plt.figure(figsize=(4, 4))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title("")
    plt.colorbar()
    plt.grid(False)
    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.rcParams.update({'font.size': 19})
    plt.ylabel('True label')
    plt.xlabel('Predicted label\n'.format())
    plt.show()

def plot_FI(clf,columns,num):
  matplotlib.rcParams.update(matplotlib.rcParamsDefault)
  importance = clf.feature_importances_
  height=[]
  bars=[]
  for i,v in enumerate(importance):
    height.append(v)
    bars.append(columns[i])
  
  for i in range(0,len(height)):
    for j in range(0,len(height)):
      if(height[i]>height[j]):
        height[i], height[j] = height[j], height[i]
        bars[i], bars[j] = bars[j], bars[i]

  height=height[:num][::-1]
  bars=bars[:num][::-1]
  y_pos = np.arange(len(bars))
  plt.figure(figsize=(4, 3))
  plt.rcParams.update({'font.size': 10})
  plt.barh(y_pos, height)
  plt.yticks(y_pos, bars)
  plt.show()


def plot_learning_curve_model(X, Y, model,smote):
    matplotlib.rcParams.update(matplotlib.rcParamsDefault)
    cv = 5
    train_sizes=[]
    train_scores=[]
    test_scores=[]
    partitions=15
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)
    for i in range(1,partitions+1):
        percentage="Percentage: "+str(int(100*(i)/partitions))+"%"
        sys.stdout.write('\r'+percentage)
        if(i==partitions):
            Xtrain=X_train
            Ytrain=y_train
        
            Xtest=X_test
            Ytest=y_test
        else:
            Xtrain=X_train[:int(len(X_train)/partitions)*i]
            Ytrain=y_train[:int(len(y_train)/partitions)*i]
        
            Xtest=X_test[:int(len(X_test)/partitions)*i]
            Ytest=y_test[:int(len(y_test)/partitions)*i]
        try:
          if(smote):
              Xtrain_smote, Ytrain_smote = SMOTE().fit_resample(Xtrain, Ytrain)
              clf = model.fit(Xtrain_smote, Ytrain_smote)
          else:
              clf = model.fit(Xtrain, Ytrain)
          pred_train = clf.predict(Xtrain)
          train_scores.append([f1_score(Ytrain, pred_train),f1_score(Ytrain, pred_train)])
          pred_test = clf.predict(Xtest)
          test_scores.append([f1_score(Ytest, pred_test),f1_score(Ytest, pred_test)])
          train_sizes.append(len(Xtrain))
        except Exception as e: 
          print(" "+str(e))
          continue
    sys.stdout.write('\r')
    plt.figure(figsize=(7, 7))
    plt.tick_params(axis='y', labelleft='off', labelright='on')
    plt.xlabel("Training examples",fontsize=16)
    plt.ylabel("F1 Score",fontsize=16)
    train_size=np.linspace(.1, 1.0, 15)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std  = np.std(train_scores, axis=1)
    test_scores_mean  = np.mean(test_scores, axis=1)
    test_scores_std   = np.std(test_scores, axis=1)
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,train_scores_mean + train_scores_std, alpha=0.1,
                     color="tab:blue")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,test_scores_mean + test_scores_std, alpha=0.1, color="tab:orange")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="tab:blue",label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="tab:orange",label="Test score")
    plt.legend(loc="best",fontsize=14)
    plt.show()

def plot_ROC(clf,y_train,y_test):
  matplotlib.rcParams.update(matplotlib.rcParamsDefault)
  le = LabelEncoder()
  y_train=le.fit(y_train).transform(y_train)
  y_test=le.fit(y_test).transform(y_test)
  y_scores = clf.predict_proba(X_test)
  fpr, tpr, threshold = roc_curve(y_test, y_scores[:, 1])
  roc_auc = auc(fpr, tpr)
  plt.figure(figsize=(6,6))
  plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
  plt.legend(loc = 'lower right',fontsize=15)
  plt.plot([0, 1], [0, 1],'r--')
  plt.xlim([0, 1])
  plt.ylim([0, 1])
  plt.ylabel('True Positive Rate',fontsize=16)
  plt.xlabel('False Positive Rate',fontsize=16)
  plt.grid(False)
  plt.show()


DATASET ANALYSIS

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#IF YOU WANT A GRAPH, ACTIVATE "plot" VARIABLE

#LOAD DATASET
small=1      #set to 1 if you want the small dataset for a faster analysis
lenght=4000     #the lenght of small dataset

dataset=getDataset(small,lenght, 1)

numeric_attributes=["age","campaign","pdays","previous","emp.var.rate","cons.price.idx","cons.conf.idx","euribor3m","nr.employed"]
categorical_attributes=["job","marital","education","default","housing","loan","contact","poutcome"]

#CORRLATION MATRIX
plot=1
df=dataset[numeric_attributes]
if(plot):
    print("\nCORRLATION MATRIX")
    corr_df = df.corr(method='pearson')
    plt.figure(figsize=(7, 6))
    sns.set(font_scale=0.7)
    sns.heatmap(corr_df, annot=True)
    plt.show()


#SCATTER MATRIX
plot=0
if(plot):
  print("\nSCATTER MATRIX")
  sns.set(font_scale=0.7)
  g=sns.pairplot(df,plot_kws={"s": 4})
  g.fig.set_size_inches(10,10)
  plt.show()




#STATISTICS TABLE
plot=1
if(plot):
  print("\nSTATISTICS TABLE")
  data=[]
  plt.figure(figsize=(12, 4))
  for att in df.columns:
    data.append(list(getStatistic(df,att).values()))
  drawStatisticsTable(data, list(df.columns), list(getStatistic(df,"age").keys()))


#BOXPLOT and DENSITY  (numeric variables, change "att")
plot=0
numeric_attributes=["age","campaign","pdays","previous","emp.var.rate","cons.price.idx","cons.conf.idx","euribor3m","nr.employed"]
att=numeric_attributes[0]
if(plot):
    print("\nBOXPLOT "+att)
    flierprops = dict(markerfacecolor='0.5', markersize=10,linestyle='none')
    sns.set(font_scale=2)
    plt.figure(figsize=(8,7))
    sns.boxplot(x="y", y=att, linewidth=3,data=dataset, flierprops=flierprops,
                hue="y")
    plt.legend(loc='upper center')
    plt.xlabel("")
    plt.show()
if(plot):
    print("\nDENSITY "+att)
    plt.figure(figsize=(8,7))
    ax=sns.kdeplot(data=dataset, x=att, hue="y",multiple="stack")
    ax.yaxis.tick_right()
    ax.legend_.set_bbox_to_anchor((0.05, 0.95))
    ax.legend_._set_loc(2)
    plt.show()


#BAR CHARTS    (categorical variables, change "att")
plot=0
categorical_attributes=["job","marital","education","default","housing","loan","contact","poutcome"]
att=categorical_attributes[0]
if(plot):
    print("\nBAR CHARTS "+att)
    plt.figure(figsize=(15,6))
    fontsize=12
    margin=0.09
    height=4
    total = len(dataset)
    ax=sns.countplot(x=att, data=dataset, hue="y")
    ax.set(ylabel='#samples')
    plt.legend(loc='upper right')
    for p in ax.patches:
        ax.annotate('{:.2f}%'.format(100*p.get_height()/total), (p.get_x()+margin, p.get_height()+height)
                    ,fontsize=fontsize)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right",
                       fontsize=fontsize+4)
    plt.show()
if(plot):
    print("")
    wid=0.4
    plt.figure(figsize=(15, 6))
    keys,no_list,yes_list=getOccurrences(dataset,att,1,1)
    plt.xticks(rotation=40)
    plt.bar(keys, no_list,width=wid)
    plt.bar(keys, yes_list, bottom = no_list,width=wid)
    plt.show()


LOGISTIC REGRESSION

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from imblearn.over_sampling import *
from sklearn.linear_model import LogisticRegression
import sys
from datetime import datetime
from sklearn.metrics import *
import warnings
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
warnings.simplefilter(action='ignore', category=FutureWarning)


#LOAD DATASET
small=1      #set to 1 if you want to use the small dataset for a faster analysis
lenght=1000     #the lenght of small dataset
smote=1            #set to 1 if you want SMOTE
dataset=getDataset(small,lenght)
 
#ENCODING
dataset=encodingDataset(dataset)
 
#TRAIN, VAL e TEST
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, len(dataset.columns)-1].values
 
test_ratio = 0.10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio)

val_ratio = 0.15
kf = KFold(n_splits=int((1-test_ratio)/val_ratio))
kf.get_n_splits(X_train)

#TUNING
C_values=[100,10,1,0.1, 0.01]
Penalties=["l2","l1"]


best_accuracy=0
best_c=0
best_p=0
 
mi=10000 #max_iter
max_i=len(C_values)*len(Penalties)*int((1-test_ratio)/val_ratio)

i=-1
for p in Penalties:
    for c in C_values:
        clf = LogisticRegression(penalty=p,C=c,solver="saga",max_iter=mi,n_jobs=-1)
        mean_accuracies=[]
        for train_index, test_index in kf.split(X_train):# KFold Cross Val
            i=i+1
            percentage="Tuning: "+str(round(100*(i)/max_i,2))+"%"
            if(best_accuracy>0):
              percentage=percentage+"      best C = "+str(best_c)+"      best P = "+str(best_p)+"      best accuracy = "+str(round(best_accuracy,2))
            sys.stdout.write('\r'+percentage)
        
            X_train_tmp, X_val = X_train[train_index], X_train[test_index]
            y_train_tmp, y_val = y_train[train_index], y_train[test_index]
            if(smote):
              X_train_tmp, y_train_tmp = SMOTE().fit_resample(X_train_tmp, y_train_tmp)
            clf.fit(X_train_tmp, y_train_tmp)
            y_pred = clf.predict(X_val)
            mean_accuracies.append(accuracy_score(y_val,y_pred))
        acc=sum(mean_accuracies)/len(mean_accuracies)
        if(best_accuracy<acc):
            best_accuracy=acc
            best_c=c
            best_p=p
percentage="Tuning: 100.0%"+"      best C = "+str(best_c)+"      best P = "+str(best_p)+"      best accuracy = "+str(round(best_accuracy,2))
sys.stdout.write('\r'+percentage)
 
#TEST
if(smote):
  X_train, y_train = SMOTE().fit_resample(X_train, y_train)
clf = LogisticRegression(penalty=best_p,C=best_c,solver="saga",max_iter=mi,n_jobs=-1)
sys.stdout.write("\n\nTraining...")
clf.fit(X_train, y_train)
sys.stdout.write('\r'+"Prediction...")
y_pred = clf.predict(X_test)
sys.stdout.write('\r'+"Accuracy on test set is "+str(round(accuracy_score(y_test,y_pred),2)))


#PLOT CONFUSION MATRIX
print("\n\nCONFUSION MATRIX")
plot_cm(confusion_matrix(y_test, y_pred),["no","yes"])

#ROC CURVE
print("\n\nROC CURVE")
plot_ROC(clf,y_train,y_test)

#LEARNING CURVE
print("\n\nLEARNING CURVES")
clf = LogisticRegression(penalty=best_p,C=best_c,solver="saga",max_iter=mi,n_jobs=-1)
plot_learning_curve_model(X, y, clf,smote)

 K-NEAREST NEIGHBOR

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from imblearn.over_sampling import *
from sklearn.linear_model import LogisticRegression
import sys
from datetime import datetime
from sklearn.metrics import *
import warnings
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
warnings.simplefilter(action='ignore', category=FutureWarning)


#LOAD DATASET
small=1      #set to 1 if you want to use the small dataset for a faster analysis
lenght=1000     #the lenght of small dataset
smote=1            #set to 1 if you want SMOTE
dataset=getDataset(small,lenght)
 
#ENCODING
dataset=encodingDataset(dataset)
 
#TRAIN, VAL e TEST
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, len(dataset.columns)-1].values
 
test_ratio = 0.10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio)

val_ratio = 0.15
kf = KFold(n_splits=int((1-test_ratio)/val_ratio))
kf.get_n_splits(X_train)

#TUNING
accuracies = []
K_values=[]
for i in range(1,100):
  K_values.append(int(i*i*0.75)+1)
best_accuracy=0
best_k=0

max_i=len(K_values)*int((1-test_ratio)/val_ratio)

skip=0
i=-1
for k in K_values:
  skip=0
  clf = KNeighborsClassifier(n_neighbors=k,n_jobs=-1)
  mean_accuracies=[]
  for train_index, test_index in kf.split(X_train):# KFold Cross Val
    i=i+1
    percentage="Tuning: "+str(round(100*(i)/max_i,2))+"%"
    if(best_accuracy>0):
      percentage=percentage+"      best K = "+str(best_k)+"      best accuracy = "+str(round(best_accuracy,2))
    sys.stdout.write('\r'+percentage)
        
    X_train_tmp, X_val = X_train[train_index], X_train[test_index]
    y_train_tmp, y_val = y_train[train_index], y_train[test_index]
    if(smote):
      X_train_tmp, y_train_tmp = SMOTE().fit_resample(X_train_tmp, y_train_tmp)
    if(len(X_train_tmp)<k):
      skip=1
      break
    clf.fit(X_train_tmp, y_train_tmp)
    y_pred = clf.predict(X_val)
    mean_accuracies.append(accuracy_score(y_val,y_pred))
  if(skip==1):
    continue
  acc=sum(mean_accuracies)/len(mean_accuracies)
  accuracies.append(acc)
  if(best_accuracy<acc):
      best_accuracy=acc
      best_k=k
percentage="Tuning: 100.0%"+"      best K = "+str(best_k)+"      best accuracy = "+str(round(best_accuracy,2))
sys.stdout.write('\r'+percentage)

#TEST
if(smote):
  X_train, y_train = SMOTE().fit_resample(X_train, y_train)
clf = KNeighborsClassifier(n_neighbors=best_k,n_jobs=-1)
sys.stdout.write("\n\nTraining...")
clf.fit(X_train, y_train)
sys.stdout.write('\r'+"Prediction...")
y_pred = clf.predict(X_test)
sys.stdout.write('\r'+"Accuracy on test set is "+str(round(accuracy_score(y_test,y_pred),2)))


#PLOT CONFUSION MATRIX
print("\n\nCONFUSION MATRIX")
plot_cm(confusion_matrix(y_test, y_pred),["no","yes"])

#ROC CURVE
print("\n\nROC CURVE")
plot_ROC(clf,y_train,y_test)

#LEARNING CURVE
print("\n\nLEARNING CURVES")
clf = KNeighborsClassifier(n_neighbors=best_k,n_jobs=-1)
plot_learning_curve_model(X, y, clf,smote)

DECISION TREE

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from imblearn.over_sampling import *
from sklearn.linear_model import LogisticRegression
import sys
from datetime import datetime
from sklearn.metrics import *
import warnings
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.tree import *
from sklearn.ensemble import RandomForestClassifier
warnings.simplefilter(action='ignore', category=FutureWarning)


#LOAD DATASET
small=1      #set to 1 if you want to use the small dataset for a faster analysis
lenght=1000     #the lenght of small dataset
smote=1            #set to 1 if you want SMOTE
dataset=getDataset(small,lenght)
 
#ENCODING
dataset=encodingDataset(dataset)
 
#TRAIN, VAL e TEST
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, len(dataset.columns)-1].values
 
test_ratio = 0.10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio)

val_ratio = 0.15
kf = KFold(n_splits=int((1-test_ratio)/val_ratio))
kf.get_n_splits(X_train)

#TUNING
Criterions=["gini","entropy"]
Max_depth=[15,20,25,50,55,60,65,70,75]
min_samples_splits=[4,6,8,10,12,14,16,18,20]

best_accuracy=0
best_c=0
best_d=0
best_mss=0
 
max_i=len(Criterions)*len(Max_depth)*len(min_samples_splits)*int((1-test_ratio)/val_ratio)

i=-1
for c in Criterions:
    for d in Max_depth:
        for mss in min_samples_splits:
            clf = DecisionTreeClassifier(criterion=c,max_depth=d,max_features="sqrt",min_samples_split=mss)
            mean_accuracies=[]
            for train_index, test_index in kf.split(X_train):# KFold Cross Val
                i=i+1
                percentage="Tuning: "+str(round(100*(i)/max_i,2))+"%"
                if(best_accuracy>0):
                  percentage=percentage+"      best C = "+str(best_c)+"      best D = "+str(best_d)+"      best MSS = "+str(best_mss)+"      best accuracy = "+str(round(best_accuracy,2))
                sys.stdout.write('\r'+percentage)
                
                X_train_tmp, X_val = X_train[train_index], X_train[test_index]
                y_train_tmp, y_val = y_train[train_index], y_train[test_index]
                if(smote):
                  X_train_tmp, y_train_tmp = SMOTE().fit_resample(X_train_tmp, y_train_tmp)
                clf = clf.fit(X_train_tmp, y_train_tmp)
                y_pred = clf.predict(X_val)
                mean_accuracies.append(accuracy_score(y_val,y_pred))
            acc=sum(mean_accuracies)/len(mean_accuracies)
            if(best_accuracy<acc):
                best_accuracy=acc
                best_c=c
                best_d=d
                best_mss=mss     
percentage="Tuning: 100.0%"+"      best C = "+str(best_c)+"      best D = "+str(best_d)+"      best MSS = "+str(best_mss)+"      best accuracy = "+str(round(best_accuracy,2))
sys.stdout.write('\r'+percentage)
#TEST
if(smote):
  X_train, y_train = SMOTE().fit_resample(X_train, y_train)
clf = DecisionTreeClassifier(criterion=best_c,max_depth=best_d,max_features="sqrt",min_samples_split=best_mss)
sys.stdout.write("\n\nTraining...")
clf.fit(X_train, y_train)
sys.stdout.write('\r'+"Prediction...")
y_pred = clf.predict(X_test)
sys.stdout.write('\r'+"Accuracy on test set is "+str(round(accuracy_score(y_test,y_pred),2)))


#PLOT CONFUSION MATRIX
print("\n\nCONFUSION MATRIX")
plot_cm(confusion_matrix(y_test, y_pred),["no","yes"])

#ROC CURVE
print("\n\nROC CURVE")
plot_ROC(clf,y_train,y_test)

#PLOT FEATURE IMPORTANCE
print("\n\nFEATURE IMPORTANCE")
plot_FI(clf,num=15,columns=dataset.columns.values)

#LEARNING CURVE
print("\n\nLEARNING CURVES")
clf = DecisionTreeClassifier(criterion=best_c,max_depth=best_d,max_features="sqrt",min_samples_split=best_mss)
plot_learning_curve_model(X, y, clf,smote)

RANDOM FOREST

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from imblearn.over_sampling import *
from sklearn.linear_model import LogisticRegression
import sys
from datetime import datetime
from sklearn.metrics import *
import warnings
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.tree import *
from sklearn.ensemble import RandomForestClassifier
warnings.simplefilter(action='ignore', category=FutureWarning)


#LOAD DATASET
small=1      #set to 1 if you want to use the small dataset for a faster analysis
lenght=1000     #the lenght of small dataset
smote=1            #set to 1 if you want SMOTE
dataset=getDataset(small,lenght)
 
#ENCODING
dataset=encodingDataset(dataset)
 
#TRAIN, VAL e TEST
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, len(dataset.columns)-1].values
 
test_ratio = 0.10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio)

val_ratio = 0.15
kf = KFold(n_splits=int((1-test_ratio)/val_ratio))
kf.get_n_splits(X_train)

#TUNING
Criterions=["gini","entropy"]
Max_depth=[10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100]
min_samples_splits=[2,3,4,5,6,7,8,9]
n_estimators=[50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900]

best_accuracy=0;
best_c=0
best_d=0
best_mss=0
best_n=0

if(small):
    Criterions=["gini","entropy"]
    Max_depth=[50]
    min_samples_splits=[5]
    n_estimators=[150]

max_i=len(Criterions)*len(Max_depth)*len(min_samples_splits)*len(n_estimators)*int((1-test_ratio)/val_ratio)

i=-1
for c in Criterions:
    for d in Max_depth:
        for mss in min_samples_splits:
            for num in n_estimators:
                i=i+1
                clf = RandomForestClassifier(n_estimators=num,criterion=c,max_depth=d,max_features="sqrt",min_samples_split=mss,n_jobs=-1)
                mean_accuracies=[]
                for train_index, test_index in kf.split(X_train):# KFold Cross Val
                    i=i+1
                    percentage="Tuning: "+str(round(100*(i)/max_i,3))+"%"
                    if(best_accuracy>0):
                      percentage=percentage+"      best C = "+str(best_c)+"      best D = "+str(best_d)+"      best MSS = "+str(best_mss)+"      best N = "+str(best_n)+"      best accuracy = "+str(round(best_accuracy,2))
                    sys.stdout.write('\r'+percentage)
                
                    X_train_tmp, X_val = X_train[train_index], X_train[test_index]
                    y_train_tmp, y_val = y_train[train_index], y_train[test_index]
                    if(smote):
                        X_train_tmp, y_train_tmp = SMOTE().fit_resample(X_train_tmp, y_train_tmp)
                    clf = clf.fit(X_train_tmp, y_train_tmp)
                    y_pred = clf.predict(X_val)
                    mean_accuracies.append(accuracy_score(y_val,y_pred))
                acc=sum(mean_accuracies)/len(mean_accuracies)
                if(best_accuracy<acc):
                    best_accuracy=acc
                    best_c=c
                    best_d=d
                    best_mss=mss
                    best_n=num

percentage="Tuning: 100.0%"+"      best C = "+str(best_c)+"      best D = "+str(best_d)+"      best MSS = "+str(best_mss)+"      best N = "+str(best_n)+"      best accuracy = "+str(round(best_accuracy,2))
sys.stdout.write('\r'+percentage)
#TEST
if(smote):
  X_train, y_train = SMOTE().fit_resample(X_train, y_train)
clf = RandomForestClassifier(n_estimators=best_n,criterion=best_c,max_depth=best_d,max_features="sqrt",min_samples_split=best_mss,n_jobs=-1)
sys.stdout.write("\n\nTraining...")
clf.fit(X_train, y_train)
sys.stdout.write('\r'+"Prediction...")
y_pred = clf.predict(X_test)
sys.stdout.write('\r'+"Accuracy on test set is "+str(round(accuracy_score(y_test,y_pred),2)))


#PLOT CONFUSION MATRIX
print("\n\nCONFUSION MATRIX")
plot_cm(confusion_matrix(y_test, y_pred),["no","yes"])

#ROC CURVE
print("\n\nROC CURVE")
plot_ROC(clf,y_train,y_test)

#PLOT FEATURE IMPORTANCE
print("\n\nFEATURE IMPORTANCE")
plot_FI(clf,num=15,columns=dataset.columns.values)

#LEARNING CURVE
print("\n\nLEARNING CURVES")
clf = RandomForestClassifier(n_estimators=best_n,criterion=best_c,max_depth=best_d,max_features="sqrt",min_samples_split=best_mss,n_jobs=-1)
plot_learning_curve_model(X, y, clf,smote)

SUPPORT VECTOR MACHINE

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from imblearn.over_sampling import *
from sklearn.linear_model import LogisticRegression
import sys
from datetime import datetime
from sklearn.metrics import *
import warnings
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
warnings.simplefilter(action='ignore', category=FutureWarning)

gpu=0
if(gpu):#INSTALL THUNDERSVM TO RUN IN GPU (about 3 minutes for installation)
  !wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
  !ls
  !dpkg -i cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
  !ls /var/cuda-repo-9-0-local | grep .pub
  !apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
  !apt-get update
  !sudo apt-get install cuda-9.0
  !nvcc --version
  !pip install thundersvm
  from thundersvm import SVC
else:
  from sklearn.svm import SVC

#LOAD DATASET
small=1      #set to 1 if you want to use the small dataset for a faster analysis
lenght=1000     #the lenght of small dataset
smote=1            #set to 1 if you want SMOTE
dataset=getDataset(small,lenght)
 
#ENCODING
dataset=encodingDataset(dataset)
 
#TRAIN, VAL e TEST
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, len(dataset.columns)-1].values
 
test_ratio = 0.10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio)

val_ratio = 0.15
kf = KFold(n_splits=int((1-test_ratio)/val_ratio))
kf.get_n_splits(X_train)

#TUNING
C_values= [ 0.01, 0.1, 1, 10, 100, 1000 ]
Gamma_values=[0.0001, 0.001, 0.01, 0.1]

best_accuracy=0
best_c=0
best_g=0
 
max_i=len(C_values)*len(Gamma_values)*int((1-test_ratio)/val_ratio)

i=-1
for g in Gamma_values:
    for c in C_values:
        clf = SVC(C=c, gamma=g)
        mean_accuracies=[]
        for train_index, test_index in kf.split(X_train):# KFold Cross Val
            i=i+1
            percentage="Tuning: "+str(round(100*(i)/max_i,2))+"%"
            if(best_accuracy>0):
              percentage=percentage+"      best C = "+str(best_c)+"      best G = "+str(best_g)+"      best accuracy = "+str(round(best_accuracy,2))
            sys.stdout.write('\r'+percentage)
        
            X_train_tmp, X_val = X_train[train_index], X_train[test_index]
            y_train_tmp, y_val = y_train[train_index], y_train[test_index]
            if(smote):
              X_train_tmp, y_train_tmp = SMOTE().fit_resample(X_train_tmp, y_train_tmp)
            clf.fit(X_train_tmp, y_train_tmp)
            y_pred = clf.predict(X_val)
            mean_accuracies.append(accuracy_score(y_val,y_pred))
        acc=sum(mean_accuracies)/len(mean_accuracies)
        if(best_accuracy<acc):
            best_accuracy=acc
            best_c=c
            best_g=g
percentage="Tuning: 100.0%"+"      best C = "+str(best_c)+"      best G = "+str(best_g)+"      best accuracy = "+str(round(best_accuracy,2))
sys.stdout.write('\r'+percentage)
 
#TEST
if(smote):
  X_train, y_train = SMOTE().fit_resample(X_train, y_train)
clf = SVC(C=best_c, gamma=best_g,probability=True)
sys.stdout.write("\n\nTraining...")
clf.fit(X_train, y_train)
sys.stdout.write('\r'+"Prediction...")
y_pred = clf.predict(X_test)
sys.stdout.write('\r'+"Accuracy on test set is "+str(round(accuracy_score(y_test,y_pred),2)))


#PLOT CONFUSION MATRIX
print("\n\nCONFUSION MATRIX")
plot_cm(confusion_matrix(y_test, y_pred),["no","yes"])

#ROC CURVE
print("\n\nROC CURVE")
plot_ROC(clf,y_train,y_test)

#LEARNING CURVE
print("\n\nLEARNING CURVES")
clf = SVC(C=best_c, gamma=best_g)
plot_learning_curve_model(X, y, clf,smote)